In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, Flatten, Dropout
from keras.utils import to_categorical
from keras import metrics
import tensorflow as tf

2025-07-08 10:48:37.166206: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-08 10:48:37.176820: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751960917.189367   25880 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751960917.192838   25880 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751960917.202742   25880 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
# Check whether tensorflow sees the GPU
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Load the data
X = np.load("../../mel_spectrograms_train.npy")
y = np.load("../../labels_train.npy")

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [5]:
# === CNN + LSTM Model ===
model = Sequential([
    TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same')),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    Dropout(0.5),
    LSTM(128, dropout=0.3, recurrent_dropout=0.3),
    Dropout(0.3),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

# === Training ===
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=16)

model.save('model.keras')

I0000 00:00:1751960920.208292   25880 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5448 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1751960927.050947   25945 cuda_dnn.cc:529] Loaded cuDNN version 90501


748/748 ━━━━━━━━━━━━━━━━━━━━ 43s 49ms/step - accuracy: 0.6532 - loss: 0.6160 - val_accuracy: 0.5243 - val_loss: 0.7812
Epoch 2/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.7102 - loss: 0.5346 - val_accuracy: 0.9050 - val_loss: 0.2562
Epoch 3/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.8611 - loss: 0.3344 - val_accuracy: 0.9344 - val_loss: 0.1752
Epoch 4/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.9144 - loss: 0.2230 - val_accuracy: 0.9425 - val_loss: 0.1440
Epoch 5/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.9083 - loss: 0.2283 - val_accuracy: 0.9415 - val_loss: 0.1442
Epoch 6/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 36s 48ms/step - accuracy: 0.9343 - loss: 0.1678 - val_accuracy: 0.9545 - val_loss: 0.1254
Epoch 7/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 34s 46ms/step - accuracy: 0.9454 - loss: 0.1449 - val_accuracy: 0.9605 - val_loss: 0.1143
Epoch 8/10
748/748 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.9497 - loss: 0.1325 - val_accurac